In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dta3.csv']


In [2]:
dt=pd.read_csv('../input/dta3.csv')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dt=pd.get_dummies(dt)

In [4]:
dt=dt.drop(columns=['B_ELECTRONIC_AUCTION_N','PROCEDURE_RES','SECTOR_na','B_GPA_N','TYPE_OF_CONTRACT_W','CAE_TYPE_Z','AUTHORITY_COUNTRY_UK'] )

In [5]:
print(dt.columns)
len(dt)

Index(['YEAR', 'NUMBER_OFFERS', 'AWARD_EST_VALUE_EURO', 'AWARD_VALUE_EURO',
       'AUTHORITY_COUNTRY_AT', 'AUTHORITY_COUNTRY_BE', 'AUTHORITY_COUNTRY_BG',
       'AUTHORITY_COUNTRY_CH', 'AUTHORITY_COUNTRY_CY', 'AUTHORITY_COUNTRY_CZ',
       ...
       'SECTOR_93', 'SECTOR_98', 'B_GPA_Y', 'PROCEDURE_AWP', 'PROCEDURE_COD',
       'PROCEDURE_INP', 'PROCEDURE_NIC', 'PROCEDURE_NOC', 'PROCEDURE_OPE',
       'B_ELECTRONIC_AUCTION_Y'],
      dtype='object', length=145)


2875612

In [6]:
dt = dt[pd.notnull(dt['AWARD_VALUE_EURO'])]
dt = dt[dt['AWARD_VALUE_EURO'] != 0]
len(dt)



2875612

In [7]:
dt = dt[pd.notnull(dt['AWARD_EST_VALUE_EURO'])]
dt = dt[dt['AWARD_EST_VALUE_EURO'] != 0]
len(dt)



1476395

In [8]:
dt = dt[pd.notnull(dt['NUMBER_OFFERS'])]
dt = dt[dt['NUMBER_OFFERS'] != 0]
len(dt)

1433056

In [9]:
import numpy as np
dt['LNRATIO']=(np.log(dt['AWARD_VALUE_EURO']/dt['AWARD_EST_VALUE_EURO']))
len(dt)


1433056

In [10]:
#log of 'df_train['month'] = pd.to_datetime(df_train.Date).dt.month
dt['LN_AWARD_VALUE_EURO'] = np.log(dt['AWARD_VALUE_EURO'])
dt['LN_AWARD_EST_VALUE_EURO'] = np.log(dt['AWARD_EST_VALUE_EURO'])

In [11]:
dt2=dt

In [12]:
dt = dt[(dt.LNRATIO>-1.386) & (dt.LNRATIO<0.642)]


In [13]:
train=dt[(dt.YEAR<2017)]
test=dt[(dt.YEAR==2017)]

In [14]:
y_train = train['LNRATIO']
X_train = train.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO', 'LNRATIO', 'LN_AWARD_VALUE_EURO'], axis=1)
y_test = test['LNRATIO']
X_test = test.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO',  'LNRATIO' ,'LN_AWARD_VALUE_EURO'], axis=1)


In [15]:
import math
from scipy.stats import spearmanr
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    print('Train rmse '+str(rmse(m.predict(X_train), y_train))) 
    print('Test  rmse '+str(rmse(m.predict(X_test), y_test)))
    print('Train '+str(spearmanr(m.predict(X_train),y_train)))
    print('Test  '+str(spearmanr(m.predict(X_test),y_test)))

In [16]:
import numpy as np
import pandas as pd

from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

import xgboost as xgb

import lightgbm as lgbm

In [17]:
def objective(params,X,y):
    params = {
              'n_estimators': int(params['n_estimators']),
               'max_depth': int(params['max_depth']),
               'num_leaves':int(params['num_leaves']),
               'subsample': '{:.3f}'.format(params['subsample']),     
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree'])
    }
    
    clf = lgbm.LGBMRegressor(
        learning_rate=0.1,
        **params
    )
    
    score = cross_val_score(clf, X, y, scoring='neg_mean_squared_error', cv=2).mean()
    print("Gini {:.5f} params {}".format(score, params))
    return score
hyperopt_objective = lambda params: (-1)*objective(params, X_train, y_train)
sp = {
        'n_estimators': 1000+ hp.randint('n_estimators', 1000),
               'max_depth':  hp.choice('max_depth', [-1,4, 8, 16 ]),
               'num_leaves': hp.choice('num_leaves', [2,10, 31, 63, 127]),
               'subsample': hp.uniform('subsample', 0.6, 1.0),
               'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)
    }


best = fmin(fn=hyperopt_objective,
            space=sp,
            algo=tpe.suggest,
            max_evals=50)

Gini -0.07530 params {'n_estimators': 1186, 'max_depth': 16, 'num_leaves': 63, 'subsample': '0.608', 'colsample_bytree': '0.709'}
Gini -0.07477 params {'n_estimators': 1842, 'max_depth': 4, 'num_leaves': 63, 'subsample': '0.879', 'colsample_bytree': '0.677'}
Gini -0.07624 params {'n_estimators': 1547, 'max_depth': 16, 'num_leaves': 2, 'subsample': '0.822', 'colsample_bytree': '0.869'}
Gini -0.07624 params {'n_estimators': 1784, 'max_depth': 4, 'num_leaves': 2, 'subsample': '0.705', 'colsample_bytree': '0.894'}
Gini -0.07473 params {'n_estimators': 1818, 'max_depth': 4, 'num_leaves': 127, 'subsample': '0.790', 'colsample_bytree': '0.724'}
Gini -0.07580 params {'n_estimators': 1003, 'max_depth': 16, 'num_leaves': 127, 'subsample': '0.602', 'colsample_bytree': '0.606'}
Gini -0.07668 params {'n_estimators': 1791, 'max_depth': 16, 'num_leaves': 63, 'subsample': '0.951', 'colsample_bytree': '0.901'}
Gini -0.07624 params {'n_estimators': 1858, 'max_depth': 16, 'num_leaves': 2, 'subsample': '0

In [18]:
best_params = space_eval(sp, best)
print("BEST PARAMETERS: " + str(best_params))

BEST PARAMETERS: {'colsample_bytree': 0.6514488804199975, 'max_depth': 8, 'n_estimators': 1046, 'num_leaves': 10, 'subsample': 0.7456398460691518}


In [19]:

from sklearn.metrics import r2_score
est = lgbm.LGBMRegressor(**best_params) 
# Fit    
est.fit(X_train, y_train)
y_pred = est.predict(X_test)
 
# Predict
score = rmse(y_test, y_pred)

print("RMSE SCORE ON TEST DATA: {}".format(score))
r2a = r2_score(y_test, y_pred)
print("RMSE SCORE ON TEST DATA: {}".format(score))

RMSE SCORE ON TEST DATA: 0.2601090504493204
RMSE SCORE ON TEST DATA: 0.2601090504493204
